## How to get IRS 900 Data - Author: Hang (Jessie) Le

There are some links that I found useful to understand about the data set and get the data from AWS
1. [AWS Data](https://s3.amazonaws.com/irs-form-990): This is the main links that contains all XML files on AWS S3 bucket
2. [Github tutorial](https://github.com/lecy/Open-Data-for-Nonprofit-Research): This link provides some guidance on how to get the data set with R
3. [The IRS 990 e-file dataset: getting to the chocolatey center of data deliciousness](https://appliednonprofitresearch.com/posts/2018/06/the-irs-990-e-file-dataset-getting-to-the-chocolatey-center-of-data-deliciousness/): This link provides a overall understanding of XML and Xpath
4. [Open Data for Nonprofit Research](https://nonprofit-open-data-collective.github.io/overview/): This link provides a collection of useful links to get data dictionary and how to get data using different programming languages. 
5. [XML to Pandas dataframe](https://medium.com/@robertopreste/from-xml-to-pandas-dataframes-9292980b1c1c): This link provides guidances on transforming XML files into pandas dataframe for Python users.

### Load Packages

In [1]:
# Importing all libraries for data processing
import os
import pandas as pd 
import csv 
import requests 
import xml.etree.ElementTree as ET
from lxml import etree
pd.options.display.max_columns = None
pd.options.display.max_rows = None

#### Getting all the index files
We first want to see a list of the annual index files. Each of these files contains a list of all the e-filings available that year. I already used the result from a tutorial to get the overview about how many years that are available to download. You can check it [here](http://social-metrics.org/irs-990-e-file-data-part-1/)
![alt text](Index.PNG)
There are both CSV and JSON format versions of each annual file. You can download the csv files of index data for specific year directly from your browser using this url: https://s3.amazonaws.com/irs-form-990/index_2014.csv. If you want to get index for another year, just modify the last part of the file. For example: index for 2015 will be https://s3.amazonaws.com/irs-form-990/index_2015.csv

We will use csv index files for reference while for data collecting process, json files will be used. 

In [2]:
# Importing required libraries for getting the data
import requests 
import json
url = 'https://s3.amazonaws.com/irs-form-990/index_2014.json' # Passing the url for the specific year you want to collect.
                                                              # In this example, I'm collecting data for 2014. 
f = requests.get(url)
data = f.json()

The data returned are in JSON format and, specifically, are nested under a Filings2014 key. The following three lines of code show, respectively, the top-level 'keys' for the data, the number of objects (filings) listed under that major key, and the first index filing. We see there are 387,529  filings in the 2014 index file.

In [4]:
print('Top-level key(s) in the 2014 index file:', data.keys(), '\n')
print('# of filings in the 2014 index file:', len(data['Filings2014']), '\n')
data['Filings2014'][0] # Checking the first row in index file

Top-level key(s) in the 2014 index file: dict_keys(['Filings2014']) 

# of filings in the 2014 index file: 387529 



{'EIN': '760740805',
 'TaxPeriod': '201406',
 'DLN': '93492311012114',
 'FormType': '990EZ',
 'URL': 'https://s3.amazonaws.com/irs-form-990/201413119349201211_public.xml',
 'OrganizationName': 'ROYAL HIGH BAND BOOSTERS',
 'SubmittedOn': '2014-12-08',
 'ObjectId': '201413119349201211',
 'LastUpdated': '2016-03-21T17:23:53'}

### Collect all XML files

The next part will be collecting all XML files for a specific year and storing those files in your local machine. I suggest creating different folders for different years to store these XML files because the following code will go to specific folder to convert these XML files to csv. 

**Notice**: This process will take a long time to run so you should expect to run your computer for the entire 36 to 48 hours. You shouldn't let the computer sleep or shut down because it will terminate the connection to AWS server and hence, the process will stop. 

In [5]:
import time
for item in data['Filings2014']:
    # url of rss feed
    url = item['URL']
    xmlName = url.replace('https://s3.amazonaws.com/irs-form-990/','')
    # creating HTTP response object from given url
    resp = ''
    while resp == '':
        try:
            resp = requests.get(url)
            break
        except:
            time.sleep(10)
    # saving the xml file to your local machine
    with open('C:\\Users\\hangl\\Documents\\Summer RA\\Non-profit\\IRS\\XML_2014\\'+xmlName, 'wb') as f:
        f.write(resp.content)

### Looping through all XML files to convert to csv

#### Creating different functions

For this step, I will create different function to collect information stored in XML files. 

1. The first function ***getIRS900Info*** will specify the node to find information in ReturnHeader and ReturnData part of XML files. 

2. The second function ***getChildNodeForReturnHeader*** will check if there is any infomation under ReturnHeader section in the SML file. If yes, it will get the data and store in python dictionary.

2. The third function ***getChildNodeForReturnData*** will check if there is any infomation under ReturnData section in the XML file. If yes, it will get the data and store in python dictionary.

In [38]:
def getIRS990Info(xmlFileName,dictData,schedule):
    tree = etree.parse(xmlFileName)
    root = tree.getroot()
    # get data from ReturnHeader
    pNode = root.find('name:ReturnHeader',sitename)
    getChildNodeForReturnHeader(pNode,dictData)
    # get data from ReturnData
    pNode = root.find('name:ReturnData/name:'+schedule,sitename)
    getChildNodeForReturnData(pNode,dictData,schedule)

In [39]:
def getChildNodeForReturnHeader(node,dictdata):
    if node == None:
        return
    #if node has not children, add node to dictionary
    if len(node.getchildren()) == 0 :
        tag = node.tag.replace(siteaddress,'')
        text = ''
        if node.text != None :
            text = str(node.text.encode('utf8')).replace('b','').replace("'","")
        dictdata.update({tag : text})
        return
    #if node has children continue find child node
    else:
        for child in node:
            getChildNodeForReturnHeader(child,dictdata)

In [40]:
def getChildNodeForReturnData(node,dictdata,schedualName):  
    if node == None:
        return
    #if node has not children, add node to dictionary
    if len(node.getchildren()) == 0 :
        tag = schedualName
        text = ''
        if node.text != None :
            text = str(node.text.encode('utf8')).replace('b','').replace("'","")
        dictdata.update({tag : text})
        return
    #if node has children, continue find child node
    else:
        for child in node:
            getChildNodeForReturnData(child,dictdata,schedualName+'-'+child.tag.replace(siteaddress,''))

#### Getting the content of data

For data management purposes, we will collect different parts of data and store in different files. In this part, you have to modify which parts of the data you want to collect. For example, IRS990, Schedule D, Schedule C, etc. 

To collect different parts, you have to modify **scheduleValue** in the following code.  Keys to replace:

| Part | Keywords|
| ----------- | ----------- |
| All IRS990 |IRS990 |
| Schedule A |IRS990ScheduleA |
| Schedule B |IRS990ScheduleB |
| Schedule C |IRS990ScheduleC |
| Schedule D| IRS990ScheduleD |
| Schedule H |IRS990ScheduleH |
| Schedule J | IRS990ScheduleJ |

In [49]:
# State you want to collect 990 or Schedule data 
#In case you want to collect data for Schedule D, this will be used: scheduleValue = 'IRS990ScheduleD' 
scheduleValue = 'IRS990' # Getting IRS990 data

Next, in brief, what we want to do is write code that will loop over all XML files stored in our local folder, grab those data, convert them to a dictionary, and then store them into a new csv file with which each corporation will be stored in a row and the associated data will be stored in a column. 

Because all information will be stored in different columns, you will expect > 600 columns for IRS 990, > 200 columns for SheduleD and for other schedules. You should expect the code to run from 15 to 30 mins for this step. 

In [50]:
# create empty list for news items 
siteaddress = '{http://www.irs.gov/efile}'
sitename = {'name':'http://www.irs.gov/efile'}
directory = 'C:\\Users\\hangl\\Documents\\Summer RA\\Non-profit\\IRS\\XML_2014\\' # You need to modify the folder for specific year
dictItems = []
entries = os.listdir(directory)
for entry in entries:
    dictData = {}
    getIRS990Info(directory+entry,dictData,scheduleValue)
    if len(dictData) > 0 :
        dictItems.append(dictData)

Next, we have to conver Python dictionary to normal data frame with rows and columns. 

In [46]:
df_990 = pd.DataFrame(dictItems) # Turning dictinary into pandas data frame for ScheduleD
#df_990 = pd.DataFrame(dictItems[0:150000]) # Collect the first part of data for IRS 900. You can just get rid of the # symbol to run the code
#df_990 = pd.DataFrame(dictItems[150000:387527]) # Collect the second part, You can just get rid of the # symbol to run the code

***Note***: Usually, for ScheduleD, you can convert all information stored in dictionary into data frame in python. However, for IRS 990, the amount of data is huge so it will take a very long time to convert all data into only one file. What I suggest to do is seperating them into 2 or 3 different files and then store them as seperate csv files. 

You can modify the code to convert a part of data for IRS990 using the Ptyhon index. For example, there are 387,526 fillings for 2014 (the numbers of fillings in the index indicates 387,529 fillings but there are 3 duplicates so the final number is 387,526 fillings), we should expect the index run from 0 to 387,527. You can modify to code to convert data into 2 files: from index 0  to 150,000 and from index 150,000 - 387,527. 

I named the data frame with the same name so you don't get confused when creating too much dataframe. 

#### Checking data

To make sure we collect all data correctly, let's check the data dimensions using shape function in Python. The first value is the number of rows and the second value is the number of columns. 

For example, the second part of IRS 900 - 2014 has 237,526 rows and 1251 columns. 

In [55]:
df_990.shape

(237526, 1251)

You can also check the missing values in each columns in the data frame using this code

In [21]:
df_990.isnull().sum()

ReturnTs                                                                                   0
TaxPeriodEndDt                                                                             0
PreparerFirmEIN                                                                        38036
BusinessNameLine1Txt                                                                       0
AddressLine1Txt                                                                            0
CityNm                                                                                     0
StateAbbreviationCd                                                                       23
ZIPCd                                                                                     23
ReturnTypeCd                                                                               0
TaxPeriodBeginDt                                                                           0
EIN                                                                   

The final step is to export to csv files. 

In [56]:
df_990.to_csv('C:\\Users\\hangl\\Documents\\Summer RA\\Non-profit\\IRS\\2014_IRS990_part2.csv', index= False)

### Collecting ReturnVersion and EIN

As different version of tax return will include different columns names, this code will get you the EIN and ReturnVersion and store in csv files so that you can merge with current data.

In [30]:
sitename = {'name':'http://www.irs.gov/efile'}
directory = 'C:\\Users\\hangl\\Documents\\Summer RA\\Non-profit\\IRS\\XML_2018\\' # Stating the year and directory where you store XML files
listReturnVersionEIN = [] # Create empty dictionary to store data
files = os.listdir(directory)
for fileName in files:
    dictData = {}
    tree = etree.parse(directory+fileName)
    root = tree.getroot()
    dictData.update({'ReturnVersion':root.attrib['returnVersion']}) # Getting ReturnVersion
    node = root.find('name:ReturnHeader/name:Filer/name:EIN',sitename)# Getting only EIN under ReturnHeader section
    text = ''
    if node.text != None :
            text = str(node.text.encode('utf8')).replace('b','').replace("'","")
    dictData.update({'EIN':text})
    listReturnVersionEIN.append(dictData)

In [32]:
df_ReturnVersionEIN = pd.DataFrame(listReturnVersionEIN) # Convert Python dictionary to Dataframe

In [33]:
# Saved as CSV file
df_ReturnVersionEIN.to_csv('C:\\Users\\hangl\\Documents\\Summer RA\\Non-profit\\IRS\\2018_ReturnVersionEIN.csv', index= False)